## 오디오 녹음 후 mat파일로 변환

In [1]:
import pyaudio
import wave
import time
import scipy.io.wavfile as wavfile
import scipy.io as sio
import os

CHUNK = 12800
RATE = 12800
isStop = False

p = pyaudio.PyAudio()

output_folder = '/home/soda/Project/python/notebook/AI_Team3'
output_wav_folder = os.path.join(output_folder, 'Input_wav')
output_mat_folder = os.path.join(output_folder, 'Input_mat')

os.makedirs(output_wav_folder, exist_ok=True)
os.makedirs(output_mat_folder, exist_ok=True)

def get_unique_filename():
    current_time = time.strftime("%Y%m%d%H%M%S%f")
    return f"recording_{current_time}.wav"

filename = os.path.join(output_wav_folder, get_unique_filename())
w = wave.open(filename, 'wb')
w.setsampwidth(p.get_sample_size(pyaudio.paInt16))
w.setnchannels(1)
w.setframerate(RATE)

def callback(in_data, frame_count, time_info, status):
    w.writeframes(in_data)
    return (in_data, pyaudio.paContinue if not isStop else pyaudio.paComplete)

stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=callback)
stream.start_stream()

try:
    print("녹음 중..", end='')
    while stream.is_active():
        print(".", end="", flush=True)
        time.sleep(0.5)
except KeyboardInterrupt:
    isStop = True
    time.sleep(1)  # 녹음이 완료될 때까지 대기

stream.stop_stream()
stream.close()
w.close()
p.terminate()

# WAV 파일을 .mat 파일로 변환하는 함수
def wav_to_mat(wav_file_path, output_mat_path, channel_list_value=0, fs_value=12800.0):
    try:
        sample_rate, wav_data = wavfile.read(wav_file_path)

        # WAV 데이터를 'audio_data'라는 키를 가진 딕셔너리로 변환합니다.
        current_time = time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.gmtime())

        data_dict = {
            '__header__': f'MATLAB 5.0 MAT-file, Platform: Win32NT, CREATED on: {current_time}',
            '__version__': '1.0',
            '__globals__': [],
            'signals': wav_data,
            'channel_list': [[channel_list_value]],  # channel_list 값을 입력받은 값으로 설정
            'fs': [[fs_value]],  # fs 값을 입력받은 값으로 설정
        }
        sio.savemat(output_mat_path, data_dict)
        print(f"\n{output_mat_path} 파일로 변환 완료!")
    except Exception as e:
        print(f"\n데이터 변환 중 오류 발생: {e}")

# WAV 파일을 .mat 파일로 변환
output_mat_file = os.path.join(output_mat_folder, f"{os.path.splitext(os.path.basename(filename))[0]}.mat")
wav_to_mat(filename, output_mat_file)


녹음 중.........
/home/soda/Project/python/notebook/AI_Team3/Input_mat/recording_20220822192230%f.mat 파일로 변환 완료!


## 모델 로드 및 class 예측

In [2]:
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

# 데이터 디렉토리 및 클래스 레이블 설정
data_dir = '/content/drive/MyDrive/1. 회전기기 고장진단 분야 데이터/Data(프로젝트 데이터)'
classes = {
    'Normal': 0,
    'Inner Race': 1,
    'Misalignment': 2,
    'Outer Race': 3,
    'Roller': 4
}

data = []
labels = []

# 파일에서 데이터 로드하여 data와 labels 리스트에 추가
for class_name, class_label in classes.items():
    class_dir = os.path.join(data_dir, class_name)
    for file in os.listdir(class_dir):
        if file.endswith('.mat'):
            mat_data = sio.loadmat(os.path.join(class_dir, file))
            signals = mat_data['signals'][0]
            data.append(signals)
            labels.append(class_label)

data = np.array(data)
labels = np.array(labels)

# 주파수 도메인 특징 추출 함수 정의
def extract_frequency_features(data):
    frequency_features = []
    for signal in data:
        # FFT를 사용하여 주파수 영역으로 변환
        fft_result = np.fft.fft(signal)
        # 주파수 영역에서 관심 있는 주파수 성분 추출
        freq_range = slice(0, 1000)
        freq_features = np.abs(fft_result[freq_range])
        frequency_features.append(freq_features)
    return np.array(frequency_features)

# 주파수 도메인 특징 추출
frequency_features = extract_frequency_features(data)

# 데이터 분할 (학습 데이터와 테스트 데이터로 분리)
X_train, X_test, y_train, y_test = train_test_split(frequency_features, labels, test_size=0.2, random_state=42)

# 데이터 스케일링 (StandardScaler 사용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(frequency_features)

# SVM 모델 생성
svm_model = SVC(kernel='linear', C=1.0)

# SVM 모델 학습
svm_model.fit(X_scaled, labels)

# Roller 데이터 디렉토리 설정
roller_dir = os.path.join() # Test Data 경로 입력

# 미래 예측할 데이터 로드하여 frequency domain 특징 추출
roller_data = []
roller_labels = []

for file in os.listdir(roller_dir):
    if file.endswith('.mat'):
        mat_data = sio.loadmat(os.path.join(roller_dir, file))
        signals = mat_data['signals'][0]
        roller_data.append(signals)

# 주파수 도메인 특징 추출
roller_frequency_features = extract_frequency_features(roller_data)

# 데이터 스케일링
roller_frequency_features_scaled = scaler.transform(roller_frequency_features)

# SVM 모델로 라벨 예측
prediction = svm_model.predict(roller_frequency_features_scaled)

# 예측된 라벨 출력
print(prediction)


## 음성 및 Led 피드백

In [7]:
import pyaudio
import wave
from pop import PwmController
import time
import threading
import os

# 불빛 깜빡임 함수
def flashleds(pwm):
    for _ in range(10):
        for j in range(0, 8):
            pwm.setChannel(j)
            pwm.setDuty(100)
        time.sleep(0.05)
        for j in range(0, 8):
            pwm.setChannel(j)
            pwm.setDuty(0)
        time.sleep(0.1)

# 음성 재생 함수
def play_sound(file_path):
    w = wave.open(file_path, "rb")
    p = pyaudio.PyAudio()

    stream = p.open(format=p.get_format_from_width(w.getsampwidth()),
                    channels=w.getnchannels(),
                    rate=w.getframerate(),
                    output=True)

    data = w.readframes(1024)
    while len(data) > 0:
        stream.write(data)
        data = w.readframes(1024)

    w.close()
    stream.stop_stream()
    stream.close()
    p.terminate()

def main():
    pwm = PwmController(0x4d)
    pwm.init()
    pwm.setChannel(-1)
    pwm.setDuty(0)
    pwm.setFreq(50)

    if prediction == 0:
        # 음성 재생 - Normal.wav
        file_path = "/home/soda/Project/python/notebook/AI_Team3/Detection_Sound/Normal.wav"
        file_path = file_path.encode('utf-8').decode()
        play_sound(file_path)

    elif prediction in [1, 2, 3, 4]:
        # 불빛 깜빡이는 스레드 시작
        led_thread = threading.Thread(target=flashleds, args=(pwm,))
        led_thread.start()

        # 음성 재생 based on the prediction value
        if prediction == 1:
            file_path = "/home/soda/Project/python/notebook/AI_Team3/Detection_Sound/Inner_Race.wav"
        elif prediction == 2:
            file_path = "/home/soda/Project/python/notebook/AI_Team3/Detection_Sound/Misalignment.wav"
        elif prediction == 3:
            file_path = "/home/soda/Project/python/notebook/AI_Team3/Detection_Sound/Outer_Race.wav"
        elif prediction == 4:
            file_path = "/home/soda/Project/python/notebook/AI_Team3/Detection_Sound/Roller.wav"

        file_path = file_path.encode('utf-8').decode()
        play_sound(file_path)

        # 불빛 깜빡이는 스레드가 끝날 때까지 기다림
        led_thread.join()

prediction=3

if __name__ == "__main__":
    main()
